In [1]:
import pandas as pd
import sqlalchemy
import pyodbc
import os
from pw import *

In [2]:
from sqlalchemy.engine import URL
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER="+pna_server+";DATABASE="+pna_db+";UID="+user+";PWD="+pwd+";Trusted_Connection=YES"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

from sqlalchemy.types import NVARCHAR
cursor = engine.connect()

In [12]:
## Set brand name.
brand = 'CT'
## Set catalog run name.
catalog = 'WN24'
## Set the start/in home date.
startdate = '01/08/2024'

# Auto-setting brand name.
if brand == "CT":
    name = "Connecting Threads (CT)"
elif brand == "KP":
    name = "Knit Picks (KP)"
elif brand == "ST":
    name = "Superior Threads (ST)"
elif brand == "WC":
    name = "We Crochet (WC)"

In [13]:
csv = pd.read_csv(r'./Processing/'+brand+'_'+catalog+'.csv', dtype = object)

In [14]:
csv

,Brand,Catalog,Page,Item
0,Connecting Threads (CT),WN24,4,B118956
1,Connecting Threads (CT),WN24,4,B118957
2,Connecting Threads (CT),WN24,4,B118958
3,Connecting Threads (CT),WN24,4,B118959
4,Connecting Threads (CT),WN24,4,B118160
...,...,...,...,...
593,Connecting Threads (CT),WN24,46,B119020
594,Connecting Threads (CT),WN24,46,B119015
595,Connecting Threads (CT),WN24,46,B20950
596,Connecting Threads (CT),WN24,46,B21164


In [38]:
items = (
"""
SELECT 
    itemid,
    id as internalid
FROM DATA_IMPORTS.ns.Item
"""
)

netsuite = pd.read_sql(items, engine)

netsuite['internalid'] = netsuite['internalid'].astype(str)

In [39]:
df = csv.merge(netsuite, left_on='Item', right_on='itemid', how='inner').reset_index(drop=True)

In [40]:
# Create a list of counts with 6-digit places depending on file size.
total_rows = len(df)
counts = [str(i).zfill(4) for i in range(1, total_rows + 1)]

# Create External ID column.
df['externalid'] = brand + '-' + catalog + '-PG-' +pd.Series(counts)

# Add in home date.
df['inhome'] = startdate

# Add catalog code.
df['catalog'] = catalog

# Add brand name.
df['brand'] = name

# Drop columns
df = df.drop(['Item', 'itemid', 'Brand', 'Catalog'], axis=1)

In [42]:
# Create US mail files, put in applicable folders.
folder_path = os.path.join('Exports', brand, catalog, 'Pagination')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_path = os.path.join(folder_path, f'{brand}_{catalog}.csv')

df.to_csv(file_path, index=False)

In [23]:
# Find missing or typo itemidss.
left_join_df = csv.merge(netsuite, left_on='Item', right_on='itemid', how='left')
# Filter to get only rows where 'itemid' is null
missing_df = left_join_df[left_join_df['itemid'].isnull()]